# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)
figure_layout = {'width': '400px',
                'height': '400px',
                'border': '1px solid black',
                'padding': '1px'}

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_weather_file = "output_data/cities.csv"

# Read the cities data:

cities_weather_df = pd.read_csv(cities_weather_file)

cities_weather_df


,city,City ID,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Temperature at Date,Max Temp to Date,Wind Speed
0,San Miguel de Tucumán,3836873,AR,-26.82,-65.22,1603522973,90,73,24.00,24.00,1.00
1,Haines Junction,5969025,CA,60.75,-137.51,1603522974,100,83,-6.48,-6.48,0.61
2,Mataura,6201424,NZ,-46.19,168.86,1603522974,100,55,18.89,18.89,4.47
3,Ushuaia,3833367,AR,-54.80,-68.30,1603522974,75,80,4.00,4.00,2.10
4,Bredasdorp,1015776,ZA,-34.53,20.04,1603522974,99,88,20.00,20.00,2.10
...,...,...,...,...,...,...,...,...,...,...,...
562,Fare,4034496,PF,-16.70,-151.02,1603523112,100,68,26.74,26.74,2.27
563,Homer,5864145,US,59.64,-151.55,1603523113,90,93,5.00,5.00,3.26
564,Dongsheng,1812256,CN,39.82,109.98,1603523113,55,36,13.49,13.49,6.05
565,Pryvillya,696218,UA,49.02,38.30,1603523113,100,83,14.39,14.39,4.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store Latitude and Longitude into locations

locations = cities_weather_df[["Latitude", "Longitude"]].astype(float)

# Humidity

humidity_cities = cities_weather_df["Humidity"].astype(float)

#Humidity heatmap layer

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_cities,
                                dissipating=False, max_intensity=100,
                                point_radius = 4)

fig.add_layer(heat_layer)

fig



Figure(layout=FigureLayout(height='420px'))

In [4]:
# Humidity

humidity_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)', 
                                   stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
                                   info_box_content=[f"Humidity: {humidity}" for humidity in humidity_cities])

fig = gmaps.figure()
fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
cities_weather_df

,city,City ID,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Temperature at Date,Max Temp to Date,Wind Speed
0,San Miguel de Tucumán,3836873,AR,-26.82,-65.22,1603522973,90,73,24.00,24.00,1.00
1,Haines Junction,5969025,CA,60.75,-137.51,1603522974,100,83,-6.48,-6.48,0.61
2,Mataura,6201424,NZ,-46.19,168.86,1603522974,100,55,18.89,18.89,4.47
3,Ushuaia,3833367,AR,-54.80,-68.30,1603522974,75,80,4.00,4.00,2.10
4,Bredasdorp,1015776,ZA,-34.53,20.04,1603522974,99,88,20.00,20.00,2.10
...,...,...,...,...,...,...,...,...,...,...,...
562,Fare,4034496,PF,-16.70,-151.02,1603523112,100,68,26.74,26.74,2.27
563,Homer,5864145,US,59.64,-151.55,1603523113,90,93,5.00,5.00,3.26
564,Dongsheng,1812256,CN,39.82,109.98,1603523113,55,36,13.49,13.49,6.05
565,Pryvillya,696218,UA,49.02,38.30,1603523113,100,83,14.39,14.39,4.29


In [6]:
perfect_city_df = cities_weather_df.loc[(cities_weather_df["Max Temp to Date"] >= 25)
                                        & (cities_weather_df["Max Temp to Date"] <= 30)
                                        & (cities_weather_df["Cloudiness"] == 0)
                                        & (cities_weather_df["Wind Speed"] <= 10)]
                
perfect_city_df = perfect_city_df.dropna(how='any')

perfect_city_df.reset_index(inplace=True)

del perfect_city_df['index']

len(perfect_city_df)

perfect_city_df

,city,City ID,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Temperature at Date,Max Temp to Date,Wind Speed
0,Busselton,2075265,AU,-33.65,115.33,1603522986,0,21,25.56,25.56,1.63
1,Kidal,2455290,ML,18.44,1.41,1603522993,0,16,26.83,26.83,6.82
2,Goundam,2456917,ML,16.41,-3.67,1603523003,0,14,29.04,29.04,3.01
3,Hithadhoo,1282256,MV,-0.60,73.08,1603523003,0,75,28.67,28.67,4.93
4,Ikalamavony,1064258,MG,-21.15,46.58,1603523008,0,22,28.92,28.92,3.18
5,Abu Samrah,172515,SY,35.30,37.18,1603523014,0,15,28.82,28.82,2.63
6,Liwale,155921,TZ,-9.77,37.93,1603523014,0,45,29.51,29.51,2.54
7,Road Town,3577430,VG,18.42,-64.62,1603523018,0,85,28.15,28.33,4.47
8,Wesselsbron,940424,ZA,-27.85,26.37,1603523019,0,23,29.55,29.55,2.16
9,Marzuq,72181,YE,14.40,46.47,1603523019,0,16,25.16,25.16,3.54


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = []

# Latitude and Longitude

for index, row in perfect_city_df.iterrows():
    
    lat = row["Latitude"]
    lon = row["Longitude"]
    
    # parameters
    
    target_coordinates = f"{lat},{lon}"
    target_radius = 5000
    target_type = "hotel"
    
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base_url
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    hotel_data = response.json()

    #print(json.dumps(hotel_data, indent=4, sort_keys=True))
    
    try:
        hotel_df.append(hotel_data['results'][0]['name'])
    
    except:
        hotel_df.append("") 

perfect_city_df["First Hotel Name"] = hotel_df

perfect_city_df = perfect_city_df.dropna()

perfect_city_df

,city,City ID,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Temperature at Date,Max Temp to Date,Wind Speed,First Hotel Name
0,Busselton,2075265,AU,-33.65,115.33,1603522986,0,21,25.56,25.56,1.63,Busselton
1,Kidal,2455290,ML,18.44,1.41,1603522993,0,16,26.83,26.83,6.82,Kidal
2,Goundam,2456917,ML,16.41,-3.67,1603523003,0,14,29.04,29.04,3.01,Goundam
3,Hithadhoo,1282256,MV,-0.60,73.08,1603523003,0,75,28.67,28.67,4.93,Addu City
4,Ikalamavony,1064258,MG,-21.15,46.58,1603523008,0,22,28.92,28.92,3.18,Ikalamavony
5,Abu Samrah,172515,SY,35.30,37.18,1603523014,0,15,28.82,28.82,2.63,Suruj
6,Liwale,155921,TZ,-9.77,37.93,1603523014,0,45,29.51,29.51,2.54,Liwale
7,Road Town,3577430,VG,18.42,-64.62,1603523018,0,85,28.15,28.33,4.47,Road Town
8,Wesselsbron,940424,ZA,-27.85,26.37,1603523019,0,23,29.55,29.55,2.16,Wesselsbron
9,Marzuq,72181,YE,14.40,46.47,1603523019,0,16,25.16,25.16,3.54,Al Hujayrah


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{First Hotel Name}</dd>
<dt>city</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in perfect_city_df.iterrows()]
locations = perfect_city_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))

In [10]:
# Store Latitude and Longitude into locations

locations = cities_weather_df[["Latitude", "Longitude"]].astype(float)

# Humidity

humidity_cities = cities_weather_df["Humidity"].astype(float)

#Humidity heatmap layer

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_cities,
                                dissipating=False, max_intensity=100,
                                point_radius = 4)

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig



Figure(layout=FigureLayout(height='420px'))